In [1]:
# Import the required libraries.
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

#from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:

tf.keras.backend.clear_session()
seed_constant = 27
np.random.seed(seed_constant)
tf.random.set_seed(seed_constant)
random.seed(seed_constant)
tf.keras.utils.set_random_seed(seed_constant)

In [3]:
# Create a Matplotlib figure and specify the size of the figure.
plt.figure(figsize = (20, 20))

# Get the names of all classes/categories in UCF50.
TRAIN_DIR = r'kata/train'
VAL_DIR = r'kata/val'
TEST_DIR = r'kata/test'
all_classes_names = sorted(os.listdir(TRAIN_DIR))
print(all_classes_names)


['10outfinal', '1outfinal', '2outfinal', '3outfinal', '4outfinal', '5outfinal', '6outfinal', '7outfinal', '8outfinal', '9outfinal']


<Figure size 2000x2000 with 0 Axes>

In [4]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 80, 80

# Specify the directory containing the UCF50 dataset. 
#DATASET_DIR = "/newdatasetMediapipe/kata/"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST =all_classes_names
#CLASSES_LIST = ['2 Tolong', '7 permisi', '1 Maaf']

In [5]:
SEQUENCE_LENGTH = 15#getAverageSequenceLength()
print(SEQUENCE_LENGTH)

15


In [6]:
from sklearn.utils import shuffle


def getDatafromNpz(file):
    return file['data'],file['label']
dtrain = np.load('feature_train.npz')
dval = np.load('feature_val.npz')
dtest = np.load('feature_test.npz')

features_train,labels_train = getDatafromNpz(dtrain)
#cek1 = features_train[0]
#features_train, labels_train = shuffle(features_train, labels_train, random_state=0)
#cek2 = features_train[0]
#if np.array_equal(cek1,cek2):
#    print("sama")
#else:
#    print("beda")


features_test,labels_test = getDatafromNpz(dtest)
features_val,labels_val = getDatafromNpz(dval)

In [7]:
#The best
def create_convlstm_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################

    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(TimeDistributed(MaxPooling2D((2, 2)))) 
    model.add(TimeDistributed(Dropout(0.3)))
    
    model.add(TimeDistributed(Conv2D(
        32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.3)))
    
    #model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    #model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.3)))
    
    #model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    #model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))#GOAP
    #model.add(Dropout(0.4))                               
    model.add(LSTM(64, return_sequences=False))#32
    #model.add(BatchNormalization())
    model.add(Dropout(0.3))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [8]:
# Construct the required convlstm model.
convlstm_model = create_convlstm_model()

# Display the success message. 
print("Model Created Successfully!")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 15, 80, 80, 16)   448       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 15, 40, 40, 16)   0         
 tributed)                                                       
                                                                 
 dropout (Dropout)           (None, 15, 40, 40, 16)    0         
                                                                 
 time_distributed_2 (TimeDis  (None, 15, 40, 40, 32)   4640      
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 15, 20, 20, 32)   0         
 tributed)                                              

In [9]:
# Plot the structure of the contructed model.
#plot_model(convlstm_model, to_file = 'convlstm_model_structure_plot.png', show_shapes = True, show_layer_names = True)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback

from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

model_ckpt = ModelCheckpoint('MyNet.h5',monitor='val_accuracy', verbose=1,save_best_only=True )
# Compile the model and specify loss function, optimizer and metrics values to the model
adam=tf.keras.optimizers.Adam(learning_rate=0.0005)
convlstm_model.compile(loss = 'categorical_crossentropy',optimizer=adam, metrics = ["accuracy"])
#tf.keras.optimizers.Adam(lr=0.0005)
# Start training the model.
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 100, batch_size = 4, validation_data=(features_val, labels_val),
                                                     callbacks = [cb,model_ckpt])
                                                     #callbacks = [TerminateOnBaseline(monitor='val_accuracy', baseline=0.96)])

Epoch 1/100
297/297 [==============================] - ETA: 0s - loss: 2.2642 - accuracy: 0.1450
Epoch 1: val_accuracy improved from -inf to 0.22667, saving model to MyNet.h5
297/297 [==============================] - 8s 12ms/step - loss: 2.2642 - accuracy: 0.1450 - val_loss: 2.1866 - val_accuracy: 0.2267
Epoch 2/100
296/297 [============================>.] - ETA: 0s - loss: 2.0785 - accuracy: 0.2145
Epoch 2: val_accuracy improved from 0.22667 to 0.35333, saving model to MyNet.h5
297/297 [==============================] - 3s 10ms/step - loss: 2.0779 - accuracy: 0.2150 - val_loss: 1.8573 - val_accuracy: 0.3533
Epoch 3/100
294/297 [============================>.] - ETA: 0s - loss: 1.7630 - accuracy: 0.3588
Epoch 3: val_accuracy improved from 0.35333 to 0.48000, saving model to MyNet.h5
297/297 [==============================] - 3s 10ms/step - loss: 1.7638 - accuracy: 0.3575 - val_loss: 1.5847 - val_accuracy: 0.4800
Epoch 4/100
291/297 [============================>.] - ETA: 0s - loss: 1.

KeyboardInterrupt: 

In [ ]:
print(cb.logs)
print(sum(cb.logs))
np.save('my_history.npy',convlstm_model_training_history.history)

In [ ]:
history=np.load('my_history.npy',allow_pickle='TRUE').item()

In [ ]:
convlstm_model_training_history = history

In [ ]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
    '''
    This function will plot the metrics passed to it in a graph.
    Args:
        model_training_history: A history object containing a record of training and validation 
                                loss values and metrics values at successive epochs
        metric_name_1:          The name of the first metric that needs to be plotted in the graph.
        metric_name_2:          The name of the second metric that needs to be plotted in the graph.
        plot_name:              The title of the graph.
    '''
    
    # Get metric values using metric names as identifiers.
    metric_value_1 = model_training_history[metric_name_1]
    metric_value_2 = model_training_history[metric_name_2]
    
    # Construct a range object which will be used as x-axis (horizontal plane) of the graph.
    epochs = range(len(metric_value_1))

    # Plot the Graph.
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    # Add title to the plot.
    plt.title(str(plot_name))

    # Add legend to the plot.
    plt.legend()
    
    plt.savefig("output"+metric_name_1+".jpg")

In [ ]:
# Visualize the training and validation loss metrices.
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:

# Visualize the training and validation accuracy metrices.
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy') 

In [ ]:
# Evaluate the trained model.
from keras.models import load_model
convlstm_model = load_model("MyNet.h5")
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)


In [ ]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'convlstm_model___Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
convlstm_model.save(model_file_name)

In [ ]:
predict_x=convlstm_model.predict(features_test) 
rounded_predictions=np.argmax(predict_x,axis=1)

print(rounded_predictions[1])

rounded_labels=np.argmax(labels_test, axis=1)
print(rounded_labels[1])
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, rounded_predictions)
print(cm)


In [ ]:
from sklearn.metrics import classification_report
# predict 
predictions = convlstm_model.predict(features_test)#, batch_size = 4)
pred = np.argmax(predictions, axis=1)
# label
rounded_labels=np.argmax(labels_test, axis=1)

print(classification_report(rounded_labels, pred))
# precission “Berapa persen mahasiswa yang benar DO dari keseluruhan mahasiswa yang diprediksi DO?”
# recall "Berapa persen mahasiswa yang diprediksi DO dibandingkan keseluruhan mahasiswa yang sebenarnya DO?"